# Estructura de anotaciones en CANTEMIST

En cantemist-norm los datos se presentan de la siguiente manera:
- nota1.txt
- nota1.ann <- Fichero de anotación

La anotación viene en un fichero de texto por filas. Cada dos filas están organizadas de la siguiente forma:
La primera fila contiene esta información (sin header)

| indice término | categoria | start char offset | end char offset | mention string |
|----|-----|-----|-----|----|
| T1 | MORFOLOGIA_NEOPLASIA | 3332 | 3341 | Carcinoma miocrítico |

La segunda fila tiene esta información (sin header)
| indice | Annotator Notes | indice término | eCIE-O 3.1 code |
|----|----|----|----|
| #1| AnnotatorNotes | T1 | 8041/3 |

La anotación en la segunda fila se corresponde al término descrito en la primera. Para parsear estas anotaciones a un formato más legible he implementado la siguiente celda con la función `parse_cantemist_annotation()`.


In [16]:
import os
import pandas as pd


def parse_cantemist_annotation(ann_path):
    ''' Parsea anotaciones de Cantemist para hacer un dataframe estructurado.
    
        Input: Un fichero de anotación de cantemist. 
        Output: Un dataframe con la anotación estructurada.
    '''
    # Comprueba que el fichero no esté vacío
    if os.path.getsize(ann_path) == 0:
        return pd.DataFrame(columns=['term_idx', 'text', 'category', 'char_start', 'char_end', 'ICD-O Code'])
    
    # Lee el fichero de anotación
    ann = pd.read_csv(
        ann_path,
        sep = '\t',
        dtype=str,
        encoding='utf-8',
        header=None
    )

    # Parsea la parte de las coordenadas
    nota_coords = ann[ann[0].str.startswith('T')].copy()
    split = nota_coords[1].str.split(' ', expand=True)
    nota_coords = pd.concat([nota_coords[[0, 2]], split], axis=1)
    nota_coords.columns = ['term_idx', 'text', 'category', 'char_start', 'char_end']

    # Parsea las filas de anotación
    nota_ann = ann[ann[0].str.startswith('#')].copy()
    nota_ann[0] = nota_ann[0].str.replace('#', 'T')
    nota_ann = nota_ann.drop(1, axis=1)
    nota_ann.columns = ['term_idx', 'ICD-O Code']

    # Merge de las dos partes
    merged = pd.merge(nota_coords, nota_ann, on='term_idx', how='inner')

    return merged


In [2]:
data_path = "../.data/Cantemist/dev-set1/cantemist-norm/"
nota = "cc_onco853.ann"

df = parse_cantemist_annotation(data_path + nota)

print(df.head())


Empty DataFrame
Columns: [term_idx, text, category, char_start, char_end, ICD-O Code]
Index: []


# Frecuencia de términos
Ahora que sé parsear las anotaciones de Cantemist, quiero ver cuál es el término más frecuente en las notas. Para ello voy a leer todas las notas de .data/Cantemist/dev-set1/cantemist-norm/ e ir anotación por anotación contando los códigos ICD-O que aparecen.

In [3]:
import os


data_path = '../.data/Cantemist/dev-set1/cantemist-norm/'

dict_counts = {}

for nota in os.listdir(data_path):
    if nota.endswith('.ann'):
        df = parse_cantemist_annotation(data_path + nota)
        
        # Cuenta el número de veces que aparece cada código ICD-O
        code_counts = df['ICD-O Code'].value_counts()
        for code, count in code_counts.items():
            if code in dict_counts:
                dict_counts[code] += count
            else:
                dict_counts[code] = count

In [4]:
# Convierte el diccionario a un dataframe para visualizarlo mejor
counts_df = pd.DataFrame(list(dict_counts.items()), columns=['ICD-O Code', 'Count'])
counts_df = counts_df.sort_values(by='Count', ascending=False)

# Carga el mapa de códigos a términos
ecie_umls_map = pd.read_csv(
    "../.data/mappings/ICD-O-3.1-NCIt_Morphology_Mapping.txt",
    sep = '\t',
    dtype=str,
    encoding='utf-8'
)

# Añade a cada código su término preferido
counts_df = counts_df.merge(
    ecie_umls_map[ecie_umls_map['Term Type'] == 'PT'][['ICD-O Code', 'ICD-O string']],
    on='ICD-O Code',
    how='left'
)

print(counts_df.head(20))

   ICD-O Code  Count                                       ICD-O string
0      8000/6   1220                               Neoplasm, metastatic
1      8000/1    589    Neoplasm, uncertain whether benign or malignant
2      8000/3    242                                Neoplasm, malignant
3      8140/3     88                                Adenocarcinoma, NOS
4      8500/3     52                   Infiltrating duct carcinoma, NOS
5      8010/3     46                                     Carcinoma, NOS
6      8140/6     41                    Adenocarcinoma, metastatic, NOS
7      8720/3     33                            Malignant melanoma, NOS
8      8001/1     29  Tumor cells, uncertain whether benign or malig...
9      8001/3     25                             Tumor cells, malignant
10     8010/9     23                                     Carcinomatosis
11     8720/6     22                                                NaN
12     8010/6     21                         Carcinoma, metastat

Una vez tenemos el término que queremos buscar, en este caso "Adenocarcinoma, metastasic, NOS" (Código 8140/3), buscamos sus sinónimos para aumentar el texto a encontrar en las notas.

In [ ]:
def find_synonims(code):
    '''Función para encontrar los sinónimos de un código ICD-O.
        Input: código ICD-O (str).
        Output: lista de sinónimos (list of str).
    '''
    
    # Carga el mapa de códigos a términos
    ecie_umls_map = pd.read_csv(
        "../.data/mappings/ICD-O-3.1-NCIt_Morphology_Mapping.txt",
        sep = '\t',
        dtype=str,
        encoding='utf-8'
    )
    
    filtered = ecie_umls_map[(ecie_umls_map['ICD-O Code'] == code) & (ecie_umls_map['Term Type'] == 'SY')]
    
    # Si no tiene sinónimos, devuelve una lista vacía
    if filtered.empty:
        return []
    
    synonims = filtered['NCIt PT string (Preferred term)'].tolist()
    return synonims

# Ejemplo de uso
example_code = counts_df.iloc[6]['ICD-O Code']
syns = find_synonims(example_code)
print(f'Sinónimos para el código {example_code}: {syns}')

example_code = counts_df.iloc[0]['ICD-O Code']
syns = find_synonims(example_code)
print(f'Sinónimos para el código {example_code}: {syns}')

Sinónimos para el código 8140/6: []
Sinónimos para el código 8000/6: ['Secondary Neoplasm', 'Tumor Embolism', 'Metastatic Neoplasm', 'Secondary Neoplasm']
